In [1]:
import requests
import pandas as pd
import pypyodbc as odbc
import time

# --- 1️⃣ Connect to SQL Server ---
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

conn = odbc.connect(connection_string)

# --- 2️⃣ Fetch all coins from the DB ---
coins_df = pd.read_sql(
    "SELECT DISTINCT id, name FROM bronze.coin_market WHERE id IS NOT NULL", 
    conn
)

print(f"Found {len(coins_df)} coins in the database.")

# --- 3️⃣ Define function to fetch CoinGecko data ---
def connecting_coin_market_api(coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}"
    params = {
        "localization": False,  # exclude all languages except 'en'
        "sparkline": False,
        "market_data": False    # skip market data
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        print(f"Fetched {coin_id} successfully")
        return data
    except Exception as e:
        print(f"Error fetching {coin_id}: {e}")
        return None

# --- 4️⃣ Loop through coins and fetch data ---
all_data = []

for index, row in coins_df.iterrows():
    coin_id = row['id']
    result = connecting_coin_market_api(coin_id)
    if result:
        all_data.append(result)
    time.sleep(15)  # polite delay to avoid rate limits

print(f"Fetched data for {len(all_data)} coins from CoinGecko.")

# --- 5️⃣ Convert to DataFrame for parent coin table ---
coin_df = pd.json_normalize(all_data)
coin_df

C:\Users\eBay\AppData\Local\Temp\ipykernel_20792\3550357962.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coins_df = pd.read_sql(


Found 50 coins in the database.
Fetched aave successfully
Fetched avalanche-2 successfully
Fetched binance-bridged-usdt-bnb-smart-chain successfully
Fetched binancecoin successfully
Fetched bitcoin successfully
Fetched bitcoin-cash successfully
Fetched bittensor successfully
Fetched cardano successfully
Fetched chainlink successfully
Fetched coinbase-wrapped-btc successfully
Fetched crypto-com-chain successfully
Fetched dai successfully
Fetched dogecoin successfully
Fetched ethena-staked-usde successfully
Fetched ethena-usde successfully
Fetched ethereum successfully
Fetched figure-heloc successfully
Fetched hedera-hashgraph successfully
Fetched hyperliquid successfully
Fetched leo-token successfully
Fetched litecoin successfully
Fetched mantle successfully
Fetched memecore successfully
Fetched monero successfully
Fetched paypal-usd successfully
Fetched polkadot successfully
Fetched rain successfully
Fetched ripple successfully
Fetched shiba-inu successfully
Fetched solana successfully

,id,symbol,name,web_slug,asset_platform_id,block_time_in_minutes,hashing_algorithm,categories,preview_listing,public_notice,...,detail_platforms.terra-2.decimal_place,detail_platforms.terra-2.contract_address,platforms.telos,detail_platforms.telos.decimal_place,detail_platforms.telos.contract_address,detail_platforms.telos.geckoterminal_url,platforms.initia,detail_platforms.initia.decimal_place,detail_platforms.initia.contract_address,detail_platforms.initia.geckoterminal_url
0,aave,aave,Aave,aave,ethereum,0,None,"[Decentralized Finance (DeFi), Yield Farming, ...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,avalanche-2,avax,Avalanche,avalanche,None,0,None,"[Smart Contract Platform, Layer 1 (L1), Avalan...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,binance-bridged-usdt-bnb-smart-chain,bsc-usd,Binance Bridged USDT (BNB Smart Chain),binance-bridged-usdt-bnb-smart-chain,binance-smart-chain,0,None,"[BNB Chain Ecosystem, Bridged USDT, Bridged-To...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,binancecoin,bnb,BNB,bnb,None,0,None,"[Smart Contract Platform, Exchange-based Token...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bitcoin,btc,Bitcoin,bitcoin,None,10,SHA-256,"[Smart Contract Platform, Layer 1 (L1), FTX Ho...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,bitcoin-cash,bch,Bitcoin Cash,bitcoin-cash,None,0,SHA-256,"[Smart Contract Platform, Layer 1 (L1), Bitcoi...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,bittensor,tao,Bittensor,bittensor,None,0,None,"[Artificial Intelligence (AI), Smart Contract ...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,cardano,ada,Cardano,cardano,None,0,None,"[Smart Contract Platform, Layer 1 (L1), Cardan...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,chainlink,link,Chainlink,chainlink,ethereum,0,None,"[Artificial Intelligence (AI), Business Servic...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,coinbase-wrapped-btc,cbbtc,Coinbase Wrapped BTC,coinbase-wrapped-btc,ethereum,0,None,"[Crypto-Backed Tokens, Tokenized BTC, Wrapped-...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
dev_rows = []
for coin in all_data:
    dev = coin.get('developer_data', {})
    dev['coin_id'] = coin.get('id')  # <-- attach parent coin ID
    dev_rows.append(dev)

# Create a DataFrame
developer_df = pd.DataFrame(dev_rows)
developer_df

,forks,stars,subscribers,total_issues,closed_issues,pull_requests_merged,pull_request_contributors,code_additions_deletions_4_weeks,commit_count_4_weeks,last_4_weeks_commit_activity_series,coin_id
0,583,1439,104,0,0,4,2,"{'additions': 0, 'deletions': 0}",0,[],aave
1,655,2100,127,767,629,1544,75,"{'additions': 5437, 'deletions': -4061}",50,[],avalanche-2
2,0,0,0,0,0,0,0,"{'additions': None, 'deletions': None}",0,[],binance-bridged-usdt-bnb-smart-chain
3,2055,4225,470,0,0,55,26,"{'additions': 0, 'deletions': 0}",0,[],binancecoin
4,36426,73168,3967,7743,7380,11215,846,"{'additions': 1570, 'deletions': -1948}",108,[],bitcoin
5,222,321,48,0,0,454,57,"{'additions': 0, 'deletions': 0}",0,[],bitcoin-cash
6,420,1307,40,0,0,1939,117,"{'additions': 9648, 'deletions': -5498}",0,[],bittensor
7,643,3757,465,849,665,1706,80,"{'additions': 0, 'deletions': 0}",0,[],cardano
8,1538,5703,304,417,349,8258,173,"{'additions': 14978, 'deletions': -9282}",87,[],chainlink
9,0,0,0,0,0,0,0,"{'additions': None, 'deletions': None}",0,[],coinbase-wrapped-btc


In [3]:
links_rows = []
for coin in all_data:
    links = coin.get('links', {})
    links['coin_id'] = coin.get('id')  # <-- attach parent coin ID
    links_rows.append(links)

# Create a DataFrame
links_df = pd.DataFrame(links_rows)
links_df



,homepage,whitepaper,blockchain_site,official_forum_url,chat_url,announcement_url,snapshot_url,twitter_screen_name,facebook_username,bitcointalk_thread_identifier,telegram_channel_identifier,subreddit_url,repos_url,coin_id
0,[https://app.aave.com/],https://github.com/aave/aave-protocol/blob/mas...,[https://etherscan.io/token/0x7fc66500c84a76ad...,[https://www.instagram.com/aave.aave/],[https://aave.com/discord],[],https://snapshot.org/#/aave.eth,aave,AaveCom,2090735.0,Aavesome,https://www.reddit.com/r/Aave_Official,{'github': ['https://github.com/aave/aave-prot...,aave
1,[https://www.avax.network],,"[https://snowtrace.io/, https://platform.arkha...",[],"[https://discord.com/invite/avax, https://www....",[https://www.linkedin.com/company/avalancheavax],None,avax,,NaN,avalancheavax,https://www.reddit.com/r/Avax,{'github': ['https://github.com/ava-labs/avala...,avalanche-2
2,[https://bscscan.com/token/0x55d398326f99059ff...,,[https://bscscan.com/token/0x55d398326f99059ff...,[],[],[],None,,,NaN,,None,"{'github': [], 'bitbucket': []}",binance-bridged-usdt-bnb-smart-chain
3,[https://www.binance.com],,"[https://bscscan.com, https://intel.arkm.com/e...",[],[],[],None,binance,,NaN,,https://www.reddit.com/r/binance,{'github': ['https://github.com/binance-exchan...,binancecoin
4,[http://www.bitcoin.org],https://bitcoin.org/bitcoin.pdf,"[https://mempool.space/, https://platform.arkh...",[https://bitcointalk.org/],[],[],None,bitcoin,bitcoins,NaN,,https://www.reddit.com/r/Bitcoin/,{'github': ['https://github.com/bitcoin/bitcoi...,bitcoin
5,[https://bch.info/],,"[https://bch.tokenview.io/, https://www.oklink...",[],[],[],None,,,NaN,bchchannel,https://www.reddit.com/r/btc,{'github': ['https://github.com/bitcoincashorg...,bitcoin-cash
6,"[https://bittensor.com/, https://docs.bittenso...",https://bittensor.com/whitepaper,[https://taostats.io/],[],[https://discord.gg/bittensor],[],None,opentensor,,NaN,taobittensor,None,{'github': ['https://github.com/opentensor/Bit...,bittensor
7,[https://cardano.org/],https://docs.cardano.org/introduction,"[https://cardanoscan.io/, https://cardanoexplo...",[https://forum.cardano.org/],[],[],None,Cardano_CF,,NaN,Cardano,https://www.reddit.com/r/cardano,{'github': ['https://github.com/input-output-h...,cardano
8,[https://chain.link/],,[https://etherscan.io/token/0x514910771af9ca65...,[https://www.linkedin.com/company/chainlink-labs],[https://www.instagram.com/chainlinklabs],[https://blog.chain.link/],None,chainlink,,NaN,chainlinkofficial,https://www.reddit.com/r/Chainlink/,{'github': ['https://github.com/smartcontractk...,chainlink
9,"[https://www.coinbase.com/, https://coinbase.c...",https://coinbase.bynder.com/m/1303c2f4d78fc966...,[https://etherscan.io/token/0xcbb7c0000ab88b47...,[],[],[],None,coinbase,,NaN,,None,"{'github': [], 'bitbucket': []}",coinbase-wrapped-btc


In [4]:
categories_rows = []

for coin in all_data:
    coin_id = coin.get('id')
    for cat in coin.get('categories', []):  # cat is a string
        categories_rows.append({
            'coin_id': coin_id,
            'category': cat
        })

categories_df = pd.DataFrame(categories_rows)
categories_df


,coin_id,category
0,aave,Decentralized Finance (DeFi)
1,aave,Yield Farming
2,aave,BNB Chain Ecosystem
3,aave,Lending/Borrowing Protocols
4,aave,Avalanche Ecosystem
...,...,...
659,zcash,Proof of Work (PoW)
660,zcash,Pantera Capital Portfolio
661,zcash,Made in USA
662,zcash,Coinbase 50 Index


In [5]:
platforms_list = []
for coin in all_data:
    platforms = coin.get('platforms', {})
    if platforms:
        for platform, address in platforms.items():
            platforms_list.append({
                'coin_id': coin['id'],
                'platform': platform,
                'contract_address': address
            })

platforms_df = pd.DataFrame(platforms_list)
platforms_df

,coin_id,platform,contract_address
0,aave,ethereum,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
1,aave,base,0x63706e401c06ac8513145b7687a14804d17f814b
2,aave,hydration,asset_registry%2F1000624
3,aave,harmony-shard-0,0xcf323aad9e522b93f11c352caa519ad0e14eb40f
4,aave,huobi-token,0x202b4936fe1a82a4965220860ae46d7d3939bb25
...,...,...,...
327,wrapped-steth,ethereum,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0
328,wrapped-steth,monad,0x10aeaf63194db8d453d4d85a06e5efe1dd0b5417
329,wrapped-steth,unichain,0xc02fe7317d4eb8753a02c35fe019786854a92001
330,zcash,,


In [6]:
ticker_rows = []

for coin in all_data:
    coin_id = coin.get('id')
    for ticker in coin.get('tickers', []):
        # add the parent coin ID
        ticker['coin_id'] = coin_id
        ticker_rows.append(ticker)

# json_normalize to flatten nested structures
tickers_df = pd.json_normalize(
    ticker_rows,
    sep='_'
)

tickers_df


,base,target,last,volume,trust_score,bid_ask_spread_percentage,timestamp,last_traded_at,last_fetch_at,is_anomaly,...,coin_mcap_usd,market_name,market_identifier,market_has_trading_incentive,converted_last_btc,converted_last_eth,converted_last_usd,converted_volume_btc,converted_volume_eth,converted_volume_usd
0,AAVE,USDT,194.220000,2.507919e+06,green,0.010294,2025-12-15T09:29:53+00:00,2025-12-15T09:29:53+00:00,2025-12-15T09:29:53+00:00,False,...,2.951639e+09,CoinUp.io,coinup,False,0.002160,0.061702,194.22,5365.000000,153238.000000,4.823523e+08
1,AAVE,USDT,194.340000,4.031516e+05,green,0.015147,2025-12-15T09:36:37+00:00,2025-12-15T09:36:37+00:00,2025-12-15T09:37:38+00:00,False,...,2.951639e+09,Bybit,bybit_spot,False,0.002163,0.061767,194.34,863.808000,24670.000000,7.762205e+07
2,AAVE,USDT,194.330000,1.612007e+05,green,0.010285,2025-12-15T09:37:00+00:00,2025-12-15T09:37:00+00:00,2025-12-15T09:37:00+00:00,False,...,2.951639e+09,Zoomex,zoomex,False,0.002163,0.061771,194.33,345.386000,9866.000000,3.103655e+07
3,AAVE,USDT,194.420000,2.298914e+05,green,0.015148,2025-12-15T09:37:58+00:00,2025-12-15T09:37:58+00:00,2025-12-15T09:37:58+00:00,False,...,2.951639e+09,BVOX,bitvenus_spot,False,0.002164,0.061792,194.42,492.141000,14056.000000,4.422398e+07
4,AAVE,USDT,194.320000,6.627860e+04,green,0.015144,2025-12-15T09:36:25+00:00,2025-12-15T09:36:25+00:00,2025-12-15T09:38:07+00:00,False,...,2.951639e+09,Binance,binance,False,0.002163,0.061771,194.40,143.369000,4094.000000,1.288441e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3931,ZEC,EUR,363.000000,7.438016e-02,None,16.023716,2025-12-11T08:41:26+00:00,2025-12-11T08:41:26+00:00,2025-12-11T14:59:32+00:00,True,...,6.713894e+09,zondacrypto,bitbay,False,0.004759,0.134593,426.07,0.000354,0.010011,3.169000e+01
3932,SYS,ZEC,0.000047,1.046999e+02,None,3.305189,2025-12-10T03:52:43+00:00,2025-12-10T03:52:43+00:00,2025-12-11T01:54:38+00:00,True,...,1.566610e+07,Komodo Wallet,komodo_wallet,False,0.004809,0.134101,436.88,0.000024,0.000665,2.170000e+00
3933,KMD,ZEC,0.000085,1.791000e+02,None,18.858718,2025-12-09T12:54:24+00:00,2025-12-09T12:54:24+00:00,2025-12-10T04:25:33+00:00,True,...,5.894147e+06,Komodo Wallet,komodo_wallet,False,0.003994,0.111545,369.38,0.000061,0.001706,5.650000e+00
3934,ZEC,UAH,19990.001001,5.002501e-04,None,16.217986,2025-12-14T14:57:00+00:00,2025-12-14T14:57:00+00:00,2025-12-15T09:43:22+00:00,True,...,6.713894e+09,BTC Trade UA,btc_trade_ua,False,0.005267,0.150378,473.29,0.000003,0.000075,2.367640e-01


In [7]:
# Rename description.en → description
coin_df = coin_df.rename(columns={'description.en': 'description'})

# Select only relevant parent columns
parent_columns = [
    'id',
    'symbol',
    'name',
    'preview_listing',
    'description',
    'country_origin',
    'genesis_date',
    'last_updated'
]

coin_df = coin_df.reindex(columns=parent_columns)
coin_df

,id,symbol,name,preview_listing,description,country_origin,genesis_date,last_updated
0,aave,aave,Aave,False,Aave is a decentralized money market protocol ...,,None,2025-12-15T09:39:05.187Z
1,avalanche-2,avax,Avalanche,False,Avalanche is a high throughput smart contract ...,,None,2025-12-15T09:39:04.818Z
2,binance-bridged-usdt-bnb-smart-chain,bsc-usd,Binance Bridged USDT (BNB Smart Chain),False,,,None,2025-12-15T09:39:33.321Z
3,binancecoin,bnb,BNB,False,Binance Coin is the cryptocurrency of the Bina...,,2017-07-08,2025-12-15T09:39:52.013Z
4,bitcoin,btc,Bitcoin,False,Bitcoin is the world's first decentralized cry...,,2009-01-03,2025-12-15T09:40:02.516Z
5,bitcoin-cash,bch,Bitcoin Cash,False,Bitcoin Cash is a hard fork of Bitcoin with a ...,,None,2025-12-15T09:40:07.967Z
6,bittensor,tao,Bittensor,False,Bittensor is an open-source protocol that powe...,,None,2025-12-15T09:39:57.535Z
7,cardano,ada,Cardano,False,Cardano is a blockchain platform built on acad...,,None,2025-12-15T09:40:08.053Z
8,chainlink,link,Chainlink,False,Chainlink is the industry-standard decentraliz...,,2017-09-16,2025-12-15T09:40:30.320Z
9,coinbase-wrapped-btc,cbbtc,Coinbase Wrapped BTC,False,"Coinbase Wrapped BTC (""cbBTC"") is a token that...",,None,2025-12-15T09:41:26.089Z


In [ ]:
developer_df , tickers_df , categories_df , links_df, coin_df , platforms_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3945 entries, 0 to 3944
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   base                          3945 non-null   object 
 1   target                        3945 non-null   object 
 2   last                          3945 non-null   float64
 3   volume                        3945 non-null   float64
 4   trust_score                   3758 non-null   object 
 5   bid_ask_spread_percentage     3897 non-null   float64
 6   timestamp                     3945 non-null   object 
 7   last_traded_at                3945 non-null   object 
 8   last_fetch_at                 3945 non-null   object 
 9   is_anomaly                    3945 non-null   bool   
 10  is_stale                      3945 non-null   bool   
 11  trade_url                     3827 non-null   object 
 12  token_info_url                62 non-null     object 
 13  coi

,base,target,last,volume,trust_score,bid_ask_spread_percentage,timestamp,last_traded_at,last_fetch_at,is_anomaly,...,coin_mcap_usd,market_name,market_identifier,market_has_trading_incentive,converted_last_btc,converted_last_eth,converted_last_usd,converted_volume_btc,converted_volume_eth,converted_volume_usd
0,AAVE,USDT,193.700000,2.513355e+06,green,0.015505,2025-12-14T12:30:54+00:00,2025-12-14T12:30:54+00:00,2025-12-14T12:30:54+00:00,False,...,2.939371e+09,CoinUp.io,coinup,False,0.002167,0.062535,193.72,5.529000e+03,159534.000000,4.942077e+08
1,AAVE,USDT,193.870000,2.363685e+05,green,0.015162,2025-12-14T12:33:56+00:00,2025-12-14T12:33:56+00:00,2025-12-14T12:33:56+00:00,False,...,2.939371e+09,BVOX,bitvenus_spot,False,0.002168,0.062538,193.89,5.190360e+02,14969.000000,4.640990e+07
2,AAVE,USDT,193.880000,2.278007e+05,green,0.015162,2025-12-14T12:35:47+00:00,2025-12-14T12:35:47+00:00,2025-12-14T12:36:26+00:00,False,...,2.939371e+09,Bybit,bybit_spot,False,0.002168,0.062523,193.90,5.007190e+02,14442.000000,4.478857e+07
3,AAVE,USDT,193.940000,9.122622e+04,green,0.015482,2025-12-14T12:36:00+00:00,2025-12-14T12:36:00+00:00,2025-12-14T12:36:00+00:00,False,...,2.939371e+09,Zoomex,zoomex,False,0.002168,0.062542,193.96,2.005260e+02,5784.000000,1.793672e+07
4,AAVE,USDT,193.830000,4.647673e+04,green,0.015162,2025-12-14T12:34:25+00:00,2025-12-14T12:34:25+00:00,2025-12-14T12:36:18+00:00,False,...,2.939371e+09,Binance,binance,False,0.002168,0.062526,193.91,1.007530e+02,2906.000000,9.012189e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3940,ZEC,EUR,363.000000,7.438016e-02,None,16.018627,2025-12-11T08:41:26+00:00,2025-12-11T08:41:26+00:00,2025-12-11T14:59:32+00:00,True,...,7.009376e+09,zondacrypto,bitbay,False,0.004759,0.134593,426.07,3.539500e-04,0.010011,3.169000e+01
3941,SYS,ZEC,0.000047,1.046999e+02,None,1.607491,2025-12-10T03:52:43+00:00,2025-12-10T03:52:43+00:00,2025-12-11T01:54:38+00:00,True,...,1.592090e+07,Komodo Wallet,komodo_wallet,False,0.004809,0.134101,436.88,2.385000e-05,0.000665,2.170000e+00
3942,ZEC,INR,31738.300000,6.900000e-03,None,7.767628,2025-12-12T05:39:05+00:00,2025-12-12T05:39:05+00:00,2025-12-12T12:29:51+00:00,True,...,7.009376e+09,Giottus,giottus,False,0.003805,0.108444,350.81,2.626000e-05,0.000748,2.420000e+00
3943,KMD,ZEC,0.000085,1.791000e+02,None,18.552992,2025-12-09T12:54:24+00:00,2025-12-09T12:54:24+00:00,2025-12-10T04:25:33+00:00,True,...,5.892721e+06,Komodo Wallet,komodo_wallet,False,0.003994,0.111545,369.38,6.107000e-05,0.001706,5.650000e+00


In [17]:
coin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               50 non-null     object
 1   symbol           50 non-null     object
 2   name             50 non-null     object
 3   preview_listing  50 non-null     bool  
 4   description      50 non-null     object
 5   country_origin   50 non-null     object
 6   genesis_date     12 non-null     object
 7   last_updated     50 non-null     object
dtypes: bool(1), object(7)
memory usage: 2.9+ KB


In [21]:
table_name = "bronze.coin_market"
table_name_2 = "bronze.coin_basic_info"
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

connection = odbc.connect(connection_string)

cursor = connection.cursor()

params_list = [
                (
                    row['id'], row['symbol'], row['name'], row['preview_listing'], row['description'],
                    row['country_origin'], row['genesis_date'], row['last_updated']
                )
                for _, row in coin_df.iterrows()
            ]

load_query = f'''
            INSERT INTO {table_name_2} (
                id, symbol, name, preview_listing, description,
                country_origin, genesis_date, last_updated
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            '''
cursor.executemany(load_query, params_list)
connection.commit()
print(f"🆕 {len(params_list)} rows loaded for new coins succesfully..........................✅✅✅")

connection.close()
cursor.close()





🆕 50 rows loaded for new coins succesfully..........................✅✅✅


In [52]:
table_name = "bronze.coin_market"
table_name_2 = "bronze.coin_basic_info"
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

connection = odbc.connect(connection_string)

cursor = connection.cursor()


sql = (f"""
        SELECT id, Max([last_updated]) AS last_data_load
        FROM {table_name_2}
        GROUP BY id
        """
        )

cursor.execute(sql)


db_coin = cursor.fetchall()

cursor.close()

db_coin = pd.DataFrame(db_coin, columns=['coin_id','last_data_load'])


new_data_new_coin = coin_df[
    ~coin_df['id'].isin(db_coin['coin_id'])
]


old_coin_df = db_coin[
    ~db_coin['coin_id'].isin(coin_df['id'])
]

coins_to_delete = old_coin_df['coin_id'].tolist()


len(new_data_new_coin)

    
if not new_data_new_coin.empty:


    connection_string = (
        f"DRIVER={{SQL Server}};"
        f"SERVER={r'Akeelah\SQLEXPRESS'};"
        f"DATABASE=Crypto_DataWarehouse;"
        "Trusted_Connection=yes;"
    )

    connection = odbc.connect(connection_string)

    cursor = connection.cursor()

    params_list = [
                    (
                        row['id'], row['symbol'], row['name'], row['preview_listing'], row['description'],
                        row['country_origin'], row['genesis_date'], row['last_updated']
                    )
                    for _, row in new_data_new_coin.iterrows()
                ]

    load_query = f'''
                INSERT INTO {table_name_2} (
                    id, symbol, name, preview_listing, description,
                    country_origin, genesis_date, last_updated
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                '''
    cursor.executemany(load_query, params_list)
    connection.commit()
    print(f'{len(params_list)} were inserted')

else:
    print('no coin to load')

if not old_coin_df.empty:

    placeholders = ",".join("?" for _ in coins_to_delete)

    delete_sql = f"""
                    DELETE FROM {table_name_2}
                    WHERE id IN ({placeholders})
                """

    cursor.execute(delete_sql)
    deleted_rows = cursor.rowcount
    connection.commit()

    print(f"🗑️ Deleted {deleted_rows} rows for coins that are not in top 50 anymore")
else:
     print("ℹ️ No  coins to delete")






    

no coin to load
ℹ️ No  coins to delete


In [148]:
table_config = {
    "bronze.coin_basic_info": {
        "id_col": "id",
        "insert_cols": [
            "id", "symbol", "name", "preview_listing",
            "description", "country_origin",
            "genesis_date", "last_updated"
        ],
        "source_df": coin_df  # <-- use coin_df for this table
    },
    "bronze.coin_categories_info": {
        "id_col": "coin_id",
        "insert_cols": [
            "coin_id", "category"
        ],
        "source_df": categories_df  # <-- use categories_df for this table
    }
}

table_name_2 = "bronze.coin_basic_info"

connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

connection = odbc.connect(connection_string)
cursor = connection.cursor()

# Fetch existing coins from coin_basic_info
sql = f"""
    SELECT id, MAX([last_updated]) AS last_data_load
    FROM {table_name_2}
    GROUP BY id
"""
cursor.execute(sql)
db_coin = cursor.fetchall()
db_coin = pd.DataFrame(db_coin, columns=['coin_id', 'last_data_load'])

# Compute new and old coins based on coin_basic_info
new_data_new_coin = coin_df[~coin_df['id'].isin(db_coin['coin_id'])]
old_coin_df = db_coin[~db_coin['coin_id'].isin(coin_df['id'])]
coins_to_delete = old_coin_df['coin_id'].tolist()

for table, conf in table_config.items():
    source_df = conf['source_df']  # use the right dataframe per table
    id_col = conf['id_col']
    insert_cols = conf['insert_cols']

    # ---------- DELETE OLD COINS ----------
    if coins_to_delete:
        placeholders = ",".join("?" for _ in coins_to_delete)
        delete_sql = f"""
            DELETE FROM {table}
            WHERE {id_col} IN ({placeholders})
        """
        cursor.execute(delete_sql, coins_to_delete)
        deleted_rows = cursor.rowcount
        connection.commit()
        print(f'{deleted_rows} rows of data were deleted from {table}')
    else:
        print(f'No rows to delete from {table}')

    # ---------- INSERT NEW COINS ----------
    if table == table_name_2:
        insert_df = source_df[source_df[id_col].isin(new_data_new_coin['id'])]
        print(insert_df)
    else:
        # For dependent tables, only insert rows where coin_id exists in coin_basic_info
        if 'coin_id' in source_df.columns:
            insert_df = source_df[source_df['coin_id'].isin(old_coin_df['coin_id'])]
            print(insert_df)

        else:
            insert_df = source_df.copy()

    if not insert_df.empty:
        placeholders = ",".join("?" * len(insert_cols))
        params = list(insert_df[insert_cols].itertuples(index=False, name=None))

        insert_sql = f"""
            INSERT INTO {table} ({",".join(insert_cols)})
            VALUES ({placeholders})
        """
        cursor.executemany(insert_sql, params)
        connection.commit()
        print(f'{len(params)} rows inserted into {table}')
    else:
        print(f'No new rows to insert into {table}')

cursor.close()
connection.close()


No rows to delete from bronze.coin_basic_info
                                      id      symbol  \
0                                   aave        aave   
1                            avalanche-2        avax   
2   binance-bridged-usdt-bnb-smart-chain     bsc-usd   
3                            binancecoin         bnb   
4                                bitcoin         btc   
5                           bitcoin-cash         bch   
6                              bittensor         tao   
7                                cardano         ada   
8                              chainlink        link   
9                   coinbase-wrapped-btc       cbbtc   
10                      crypto-com-chain         cro   
11                                   dai         dai   
12                              dogecoin        doge   
13                    ethena-staked-usde       susde   
14                           ethena-usde        usde   
15                              ethereum         eth   
16

In [145]:
coin_df

,id,symbol,name,preview_listing,description,country_origin,genesis_date,last_updated
0,aave,aave,Aave,False,Aave is a decentralized money market protocol ...,,None,2025-12-15T09:39:05.187Z
1,avalanche-2,avax,Avalanche,False,Avalanche is a high throughput smart contract ...,,None,2025-12-15T09:39:04.818Z
2,binance-bridged-usdt-bnb-smart-chain,bsc-usd,Binance Bridged USDT (BNB Smart Chain),False,,,None,2025-12-15T09:39:33.321Z
3,binancecoin,bnb,BNB,False,Binance Coin is the cryptocurrency of the Bina...,,2017-07-08,2025-12-15T09:39:52.013Z
4,bitcoin,btc,Bitcoin,False,Bitcoin is the world's first decentralized cry...,,2009-01-03,2025-12-15T09:40:02.516Z
5,bitcoin-cash,bch,Bitcoin Cash,False,Bitcoin Cash is a hard fork of Bitcoin with a ...,,None,2025-12-15T09:40:07.967Z
6,bittensor,tao,Bittensor,False,Bittensor is an open-source protocol that powe...,,None,2025-12-15T09:39:57.535Z
7,cardano,ada,Cardano,False,Cardano is a blockchain platform built on acad...,,None,2025-12-15T09:40:08.053Z
8,chainlink,link,Chainlink,False,Chainlink is the industry-standard decentraliz...,,2017-09-16,2025-12-15T09:40:30.320Z
9,coinbase-wrapped-btc,cbbtc,Coinbase Wrapped BTC,False,"Coinbase Wrapped BTC (""cbBTC"") is a token that...",,None,2025-12-15T09:41:26.089Z


In [33]:
tickers_df

,base,target,last,volume,trust_score,bid_ask_spread_percentage,timestamp,last_traded_at,last_fetch_at,is_anomaly,...,coin_mcap_usd,market_name,market_identifier,market_has_trading_incentive,converted_last_btc,converted_last_eth,converted_last_usd,converted_volume_btc,converted_volume_eth,converted_volume_usd
0,AAVE,USDT,194.220000,2.507919e+06,green,0.010294,2025-12-15T09:29:53+00:00,2025-12-15T09:29:53+00:00,2025-12-15T09:29:53+00:00,False,...,2.951639e+09,CoinUp.io,coinup,False,0.002160,0.061702,194.22,5365.000000,153238.000000,4.823523e+08
1,AAVE,USDT,194.340000,4.031516e+05,green,0.015147,2025-12-15T09:36:37+00:00,2025-12-15T09:36:37+00:00,2025-12-15T09:37:38+00:00,False,...,2.951639e+09,Bybit,bybit_spot,False,0.002163,0.061767,194.34,863.808000,24670.000000,7.762205e+07
2,AAVE,USDT,194.330000,1.612007e+05,green,0.010285,2025-12-15T09:37:00+00:00,2025-12-15T09:37:00+00:00,2025-12-15T09:37:00+00:00,False,...,2.951639e+09,Zoomex,zoomex,False,0.002163,0.061771,194.33,345.386000,9866.000000,3.103655e+07
3,AAVE,USDT,194.420000,2.298914e+05,green,0.015148,2025-12-15T09:37:58+00:00,2025-12-15T09:37:58+00:00,2025-12-15T09:37:58+00:00,False,...,2.951639e+09,BVOX,bitvenus_spot,False,0.002164,0.061792,194.42,492.141000,14056.000000,4.422398e+07
4,AAVE,USDT,194.320000,6.627860e+04,green,0.015144,2025-12-15T09:36:25+00:00,2025-12-15T09:36:25+00:00,2025-12-15T09:38:07+00:00,False,...,2.951639e+09,Binance,binance,False,0.002163,0.061771,194.40,143.369000,4094.000000,1.288441e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3931,ZEC,EUR,363.000000,7.438016e-02,None,16.023716,2025-12-11T08:41:26+00:00,2025-12-11T08:41:26+00:00,2025-12-11T14:59:32+00:00,True,...,6.713894e+09,zondacrypto,bitbay,False,0.004759,0.134593,426.07,0.000354,0.010011,3.169000e+01
3932,SYS,ZEC,0.000047,1.046999e+02,None,3.305189,2025-12-10T03:52:43+00:00,2025-12-10T03:52:43+00:00,2025-12-11T01:54:38+00:00,True,...,1.566610e+07,Komodo Wallet,komodo_wallet,False,0.004809,0.134101,436.88,0.000024,0.000665,2.170000e+00
3933,KMD,ZEC,0.000085,1.791000e+02,None,18.858718,2025-12-09T12:54:24+00:00,2025-12-09T12:54:24+00:00,2025-12-10T04:25:33+00:00,True,...,5.894147e+06,Komodo Wallet,komodo_wallet,False,0.003994,0.111545,369.38,0.000061,0.001706,5.650000e+00
3934,ZEC,UAH,19990.001001,5.002501e-04,None,16.217986,2025-12-14T14:57:00+00:00,2025-12-14T14:57:00+00:00,2025-12-15T09:43:22+00:00,True,...,6.713894e+09,BTC Trade UA,btc_trade_ua,False,0.005267,0.150378,473.29,0.000003,0.000075,2.367640e-01
